In [41]:
import pandas as pd
from geopy.distance import geodesic
from scipy.spatial import cKDTree
from tqdm import tqdm

# Wczytanie danych
ufo_data = pd.read_csv("C:/Repos/ufo_project/data/final_file/cleared_data.csv")
airports = pd.read_csv("C:/Repos/ufo_project/data/final_file/airports_UTF-8.csv", delimiter=";")

# Filtruj tylko lotniska typu 'large_airport'
airports_large = airports[airports['type'] == 'large_airport']

# Przygotowanie danych lotnisk (współrzędne)
airports_coords = airports_large[['latitude_deg', 'longitude_deg']].values
airport_tree = cKDTree(airports_coords)  # Budowanie drzewa k-D

# Funkcja do znalezienia najbliższego lotniska
def find_nearest_airport(ufo_lat, ufo_lon):
    # Wyszukiwanie najbliższego lotniska za pomocą k-D Tree
    distance, index = airport_tree.query([ufo_lat, ufo_lon])
    nearest_airport = airports_large.iloc[index]
    return nearest_airport['name'], distance

# Przygotowanie listy do wyników
nearest_airports = []
distances = []

# Postęp w pętli
for _, ufo_event in tqdm(ufo_data.iterrows(), total=ufo_data.shape[0], desc="Przetwarzanie UFO zdarzenia"):
    ufo_lat = ufo_event['latitude']
    ufo_lon = ufo_event['longitude']
    airport, distance = find_nearest_airport(ufo_lat, ufo_lon)
    nearest_airports.append(airport)
    distances.append(distance)

# Dodanie wyników do danych UFO
ufo_data['nearest_airport'] = nearest_airports
ufo_data['distance_to_airport_km'] = distances

# Zapisanie wyniku do pliku
ufo_data.to_csv('C:/Repos/ufo_project/data/final_file/ufo_with_nearest_airports.csv', index=False)

print("Zakończono przetwarzanie. Wyniki zapisano do pliku.")




rzetwarzanie UFO zdarzenia: 100%|█████████████████████████████████████████████| 84057/84057 [00:13<00:00, 6414.31it/s]

Zakończono przetwarzanie. Wyniki zapisano do pliku.
